# Reinforcement Learning Note

---
## 一、强化学习框架：问题

---
### 1.强化学习框架（`RL框架`）
1. __RL框架__是指智能体（`Agent`）学习如何与环境互动。<br>假设时间会流逝并离散化时间步（`Time Steps`），在一开始的时间步中，只能体会观察环境，可以把这种结果看成环境呈现给智能体的情形，然后他必须选择合适的响应动作（`Action In Response`）。在下一个时间步对智能体的动作做出响应时，环境向智能体呈现新的情形，同时环境向智能体提供一个奖励，作为回应，智能体必须做出一个动作表示智能体对环境是否做出了正确的响应。这一流程继续下去，在每个时间步，环境都向智能体发送一个观察结果和奖励；作为回应，智能体必须选择一个动作。<br>通常，我们不需要假设环境向智能体显示做出合理决策所需的一切信息。但是如果这样假设，数学会大大简化，在这章节的课程中，假设智能体能够完全观察环境所处的任何状态。此时，智能体接受的不再是观察结果，而是环境状态。<img src="https://s3.cn-north-1.amazonaws.com.cn/u-img/cfff4572-b080-48ab-8872-a96d45468761" alt="" width="500px" class="index--image--1wh9w">

---
### 2.阶段性任务（`Episodic Task`）与连续性任务（`Continuing Task`）
1. 在强化学习任务中，具有清晰结束点的任务被称为，阶段性任务。将从头到尾的一系列完整互动称为一个阶段，当一个阶段结束，智能体根据获得的奖励总量，判断自己的表现，然后重头再开始，不过会带有前一次的任务记忆，使得每一次的表现越来越好。经历足够多的次数的训练后，智能体将会得出一个获得奖励非常高的策略。
2. 相应的，将没有清晰结束点的任务称为，连续性任务。智能体一直存活，需要学习选择动作的最佳方式同时与环境不断互动，

---
### 3.奖励假设（`Reward Hypothsis`）
- 在机器学习中有个重要的定义假设，即智能体的目标始终可以描述为最大化期望累计奖励（`Maximizing Expected Cumulative Reward`），这就是奖励假设。

---
### 4.动作（`Action`）、目标（`Goal`）和奖励（`Reward`）
1. 动作 - 智能体在每一个时间步根据环境，判断得出的响应；
2. 状态 - 提供给智能体的环境信息，使其能作出合理的动作；
3. 目标 - 最大化累计奖励；
4. 奖励 - 环境对智能体做出的动作的反馈，表示智能体是否做出了正确的动作。

---
### 5.折扣回报（`Discounted Return`）
1. 更改目标，更关注近期奖励，而不是遥远的未来的奖励。在随机时间步$t$，定义一个折扣率（`Discount Rate`）$\gamma \in \lbrack 0,1\rbrack$，则回报/折扣回报：$$G_{t} = R_{t+1}+\gamma R_{t+2}+\gamma^{2}R_{t+3}+\gamma^{3}R_{t+4}+\ldots = \sum_{k=0}^\infty \gamma^{k}R_{t+k+1}$$
   - 如果$\gamma = 0$，智能体只关心最即时的奖励。
   - 如果$\gamma = 1$，回报没有折扣。
   - $\gamma$的值越大，智能体越关心遥远的未来。$\gamma$的值越小，折扣程度越大，在最极端的情况下，智能体只关心最即时的奖励。
2. 使用折扣是为了避免无限未来带来的不良影响。
3. 当智能体在随机时间步选择动作时，就要用到折扣，这样程序就会更关注于获得更早出现的奖励，而不是稍后出现并且可能性较低的奖励

---
### 6.马尔可夫决策流程（`MDP - Markov Decision Process`）
1. 一步动态特性（`One-Step Dynamic Feature`）
   - 在随机时间步$t$，智能体环境互动变成一系列的状态、动作和奖励。$$(S_0, A_0, R_1, S_1, A_1, \ldots, R_{t-1}, S_{t-1}, A_{t-1}, R_t, S_t, A_t)$$当环境在时间步$t+1$对智能体做出响应时，它只考虑上一个时间步$(S_t,A_t)$的状态和动作。尤其是，它不关心再上一个时间步呈现给智能体的状态。（<em>换句话说</em>，环境不考虑任何$\lbrace S_0,\ldots,S_{t-1}\rbrace$）。<br>并且，它不考虑智能体在上个时间步之前采取的动作。（<em>换句话说</em>，环境不考虑任何$\{ A_0, \ldots, A_{t-1}\}$）。<br>此外，智能体的表现如何，或收集了多少奖励，对环境选择如何对智能体做出响应没有影响。（<em>换句话说</em>，环境不考虑任何$\{ R_0, \ldots, R_t \}$）。<br>因此，我们可以通过指定以下设置完全定义环境如何决定状态和奖励$$p(s',r|s,a) \doteq \mathbb{P}(S_{t+1}=s', R_{t+1}=r|S_t = s, A_t=a)$$对于每个可能的$s', r, s, \text{and } a$。这些条件概率用于指定环境的<strong>一步动态特性</strong>。
2. MDP
   - 一组状态$\mathcal{S}$
      - 在阶段性任务中，我们使用$\mathcal{S}^{+}$表示所有状态集合，包括终止状态。
   - 一组动作$\mathcal{A}$
   - 一组奖励$\mathcal{R}$
   - 一步动态特性$$p(s',r|s,a) \doteq \mathbb{P}(S_{t+1}=s', R_{t+1}=r|S_t = s, A_t=a)$$
   - 折扣率$\gamma \in [0,1]$
   
<img src="./definition_MDP.jpg" width="900px">

---
## 二、强化学习框架：解决方案
可以将解决方案理解为：智能体为了实现目的必须学会的一系列动作

---
### 1.策略（`Policy`）
1. __确定性策略__是从$\pi: \mathcal{S}\to\mathcal{A}$的映射。对于每个状态$s\in\mathcal{S}$，它都生成智能体在状态$s$时将选择的动作$a\in\mathcal{A}$
2. __随机性策略__是从$\pi: \mathcal{S}\times\mathcal{A}\to [0,1]$的映射。对于每个状态$s\in\mathcal{S}$和动作$a\in\mathcal{A}$，它都生成智能体在状态$s$时选择动作$a$的概率。

---
### 2.状态值函数（`State-Value Function`）
1. 策略$\pi$的__状态值函数__表示为$v_{\pi}$。对于每个状态$s \in\mathcal{S}$，它都生成智能体从状态$s$开始，然后在所有时间步根据策略选择动作的预期回报。即$$v_\pi(s) \doteq \text{} \mathbb{E}_\pi[G_t|S_t=s]$$我们将$v_\pi(s)$称之为__在策略__$\pi$__下的状态__$s$__的值__。
2. 记法$\mathbb{E}_\pi[\cdot]$来自推荐<a target="_blank" href="http://go.udacity.com/rl-textbook">教科书</a>，其中$\mathbb{E}_\pi[\cdot]$定义为随机变量的预期值（假设智能体遵守策略$\pi$）。

<img src="./definition_status_value_function.jpg" width="700px">

---
### 3.贝尔曼方程-1（`Bellman Expectation Equation`）
1. 对于一般__MDP__，我们需要使用期望值，因为通常即时奖励和下个状态无法准确地预测。因为奖励和下个状态是根据__MDP__的一步动态特性选择的。在这种情况下，奖励$r$和下个状态$s'$是从（条件性）概率分布$p(s',r|s,a)$中抽取的，__贝尔曼预期方程（对于$v_{\pi}$）__表示了任何状态$s$对于_预期即时奖励和下个状态的预期_值的值：$$v_\pi(s) = \text{} \mathbb{E}_\pi[R_{t+1} + \gamma v_\pi(S_{t+1})|S_t =s]$$
<img src="./definition_bellman_expectation_equation.jpg" width="900px">
2. __计算期望值__
   1. 如果智能体的策略$\pi$是<strong>确定性策略</strong>，智能体在状态$s$选择动作$\pi(s)$，贝尔曼预期方程可以重写为两个变量 ($s'$和$r$）的和：$$v_\pi(s) = \text{} \sum_{s'\in\mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,\pi(s))(r+\gamma  v_\pi(s'))$$在这种情况下，我们将奖励和下个状态的折扣值之和$(r+\gamma  v_\pi(s'))$与相应的概率$p(s',r|s,\pi(s))$相乘，并将所有概率相加得出预期值。
   2. 如果智能体的策略$\pi$是<strong>随机性策略</strong>，智能体在状态$s$选择动作$a$的概率是$\pi(a|s)$，贝尔曼预期方程可以重写为三个变量（$s'$、$r$和$a$）的和：$$v_\pi(s) = \text{} \sum_{s'\in\mathcal{S}^+, r\in\mathcal{R},a\in\mathcal{A}(s)}\pi(a|s)p(s',r|s,a)(r+\gamma  v_\pi(s'))$$在这种情况下，我们将奖励和下个状态的折扣值之和$(r+\gamma  v_\pi(s'))$与相应的概率$\pi(a|s)p(s',r|s,a)$相乘，并将所有概率相加得出预期值。

---
### 4.最优策略（`Optimal Policy`）
1. 策略$\pi '$定义为优于或等同于策略$\pi$（仅在所有$s\in\mathcal{S}$时$v_{\pi'}(s) \geq v_\pi(s)$）。
2. __最优策略$\pi_*$__对于所有策略$\pi$满足$\pi_* \geq \pi$。最优策略肯定存在，但并不一定是唯一的。
3. __最佳策略__正是智能体寻找的策略，是__MDP__的解决方案和实现目标的最佳策略。
4. 所有最优策略都具有相同的状态值函数$v_*$，称为__最优状态值函数__。

---
### 5.动作值函数（`Action-Value Function`）
1. 动作函数是指状态动作对$s, \pi(s)$的值是智能体从状态$s$开始并采取动作$\pi(s)$，然后遵守策略$\pi$所获得的预期回报。即$$q_\pi(s,a) \doteq \mathbb{E}_\pi[G_t|S_t=s, A_t=a]$$我们将$q_\pi(s,a)$称之为__在状态$s$根据策略$\pi$采取动作$a$的值__（或者称之为__状态动作对__$s,a$的值）。
2. 所有最优策略具有相同的动作值函数$q_*$，称之为__最优动作值函数__。
3. 对于确定性策略$\pi,v_\pi(s) = q_\pi(s, \pi(s))$适用于所有$s \in \mathcal{S}$。
4. 智能体确定最优动作值函数$q_*$后，它可以为所有$s\in\mathcal{S}$设置$$\pi_*(s) = \arg\max_{a\in\mathcal{A}(s)} q_*(s,a)$$快速获得最优策略。<br>注意，必须确保$v_*(s) = \max_{a\in\mathcal{A}(s)} q_*(s,a)$。
5. 如果在某个状态$s\in\mathcal{S}$中，有多个动作$a\in\mathcal{A}(s)$可以最大化最优动作值函数，你可以通过向任何（最大化）状态分配任意大小的概率构建一个最优策略。只需确保根据该策略给不会最大化动作值函数的动作（对于特定状态）分配的概率是 0% 即可。

---
### 6.贝尔曼方程-2
1. $q_\pi$的贝尔曼预期方程是：$$q_\pi(s,a) = \text{} \mathbb{E}_\pi[R_{t+1} + \gamma q_\pi(S_{t+1},A_{t+1}) |S_t=s,A_t=a] = \sum_{s' \in \mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,a)(r + \gamma\sum_{a' \in \mathcal{A}(s)} \pi(a'|s') q_\pi(s',a')) = \sum_{s'\in\mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,a)(r+\gamma v_\pi(s'))$$其中最后一个形式详细介绍了如何计算任意随机策略$\pi$的预期值。该方程表示任何_状态动作对_（根据任意策略）相对于_后续状态_的值（根据同一策略）的值。
2. 贝尔曼最优性方程：<br>和贝尔曼预期方程相似，贝尔曼最优性方程可以证明：状态值（以及动作值函数）满足递归关系，可以将状态值（或状态动作对的值）与所有后续状态（或状态动作对）的值联系起来。侧重于最优_策略对应的值满足的关系。
   1. __$v_*$的贝尔曼最优性方程__$$v_*(s) = \max_{a \in \mathcal{A}(s)} \mathbb{E}[R_{t+1} + \gamma v_*(S_{t+1}) | S_t=s] = \max_{a \in \mathcal{A}(s)}\sum_{s' \in \mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,a)(r + \gamma v_*(s'))$$它表示任何_状态_根据最优策略相对于_后续状态_的值（根据最优策略）的值。
   2. __$q_*$的贝尔曼最优性方程__$$q_*(s,a) = \mathbb{E}[R_{t+1} + \gamma \max_{a'\in\mathcal{A}(S_{t+1})}q_*(S_{t+1},a') | S_t=s, A_t=a]= \sum_{s' \in \mathcal{S}^+, r\in\mathcal{R}}p(s',r|s,a)(r + \gamma \max_{a'\in\mathcal{A}(s')}q_*(s',a'))$$它表示任何_状态动作对_根据最优策略相对于_后续状态动作对_（根据最优策略）的值的值。

---
## 3.动态规划